# Прекод

# Сборный проект-4

Вам поручено разработать демонстрационную версию поиска изображений по запросу.

Для демонстрационной версии нужно обучить модель, которая получит векторное представление изображения, векторное представление текста, а на выходе выдаст число от 0 до 1 — покажет, насколько текст и картинка подходят друг другу.

### Описание данных

Данные находятся [здесь](https://code.s3.yandex.net/datasets/dsplus_integrated_project_4.zip).

В файле `train_dataset.csv` собрана информация, необходимая для обучения: имя файла изображения, идентификатор описания и текст описания. Для одной картинки может быть доступно до 5 описаний. Идентификатор описания имеет формат `<имя файла изображения>#<порядковый номер описания>`.

В папке `train_images` содержатся изображения для тренировки модели.

В файле `CrowdAnnotations.tsv` — данные по соответствию изображения и описания, полученные с помощью краудсорсинга. Номера колонок и соответствующий тип данных:

1. Имя файла изображения.
2. Идентификатор описания.
3. Доля людей, подтвердивших, что описание соответствует изображению.
4. Количество человек, подтвердивших, что описание соответствует изображению.
5. Количество человек, подтвердивших, что описание не соответствует изображению.

В файле `ExpertAnnotations.tsv` содержатся данные по соответствию изображения и описания, полученные в результате опроса экспертов. Номера колонок и соответствующий тип данных:

1. Имя файла изображения.
2. Идентификатор описания.

3, 4, 5 — оценки трёх экспертов.

Эксперты ставят оценки по шкале от 1 до 4, где 1 — изображение и запрос совершенно не соответствуют друг другу, 2 — запрос содержит элементы описания изображения, но в целом запрос тексту не соответствует, 3 — запрос и текст соответствуют с точностью до некоторых деталей, 4 — запрос и текст соответствуют полностью.

В файле `test_queries.csv` находится информация, необходимая для тестирования: идентификатор запроса, текст запроса и релевантное изображение. Для одной картинки может быть доступно до 5 описаний. Идентификатор описания имеет формат `<имя файла изображения>#<порядковый номер описания>`.

В папке `test_images` содержатся изображения для тестирования модели.

## 1. Исследовательский анализ данных

Наш датасет содержит экспертные и краудсорсинговые оценки соответствия текста и изображения.

В файле с экспертными мнениями для каждой пары изображение-текст имеются оценки от трёх специалистов. Для решения задачи вы должны эти оценки агрегировать — превратить в одну. Существует несколько способов агрегации оценок, самый простой — голосование большинства: за какую оценку проголосовала большая часть экспертов (в нашем случае 2 или 3), та оценка и ставится как итоговая. Поскольку число экспертов меньше числа классов, может случиться, что каждый эксперт поставит разные оценки, например: 1, 4, 2. В таком случае данную пару изображение-текст можно исключить из датасета.

Вы можете воспользоваться другим методом агрегации оценок или придумать свой.

В файле с краудсорсинговыми оценками информация расположена в таком порядке: 

1. Доля исполнителей, подтвердивших, что текст **соответствует** картинке. 
2. Количество исполнителей, подтвердивших, что текст **соответствует** картинке.
3. Количество исполнителей, подтвердивших, что текст **не соответствует** картинке.

После анализа экспертных и краудсорсинговых оценок выберите либо одну из них, либо объедините их в одну по какому-то критерию: например, оценка эксперта принимается с коэффициентом 0.6, а крауда — с коэффициентом 0.4.

Ваша модель должна возвращать на выходе вероятность соответствия изображения тексту, поэтому целевая переменная должна иметь значения от 0 до 1.


In [235]:
import urllib.request
import zipfile
import os.path
import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

Определим список `констант` 

In [236]:
# ссылка на скачивание данных
DATA_URL='https://code.s3.yandex.net/datasets/dsplus_integrated_project_4.zip'
ROOT_FOLDER='./dsplus_integrated_project_4'
# исключаем повторную загрузку, если файл уже был ранее загружен
OUT_FILE=os.path.exists(ROOT_FOLDER + '.zip')

### Загрузка данных

In [237]:
if OUT_FILE == False:
    # скачиваем архив
    urllib.request.urlretrieve(DATA_URL, 'dsplus_integrated_project_4.zip')
    # распаковываем
    with zipfile.ZipFile('./dsplus_integrated_project_4.zip', 'r') as zip_ref:
        zip_ref.extractall(ROOT_FOLDER)

### Изучение данных

In [238]:
train_dataset_csv = pd.read_csv(ROOT_FOLDER + '/to_upload/train_dataset.csv')

train_dataset_csv.info()

train_dataset_csv.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image       5822 non-null   object
 1   query_id    5822 non-null   object
 2   query_text  5822 non-null   object
dtypes: object(3)
memory usage: 136.6+ KB


,image,query_id,query_text
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
1,1262583859_653f1469a9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
2,2447284966_d6bbdb4b6e.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
3,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
4,2621415349_ef1a7e73be.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...


#### Краудсорсинг

In [239]:
crowd_annotations_tsv = pd.read_csv(ROOT_FOLDER + '/to_upload/CrowdAnnotations.tsv', 
                                    sep='\t', 
                                    header=None, 
                                    names=['image', 'query_id', 'positive', 'n1', 'n2'])

crowd_annotations_tsv.info()

crowd_annotations_tsv.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47830 entries, 0 to 47829
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   image     47830 non-null  object 
 1   query_id  47830 non-null  object 
 2   positive  47830 non-null  float64
 3   n1        47830 non-null  int64  
 4   n2        47830 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.8+ MB


,image,query_id,positive,n1,n2
0,1056338697_4f7d7ce270.jpg,1056338697_4f7d7ce270.jpg#2,1.0,3,0
1,1056338697_4f7d7ce270.jpg,114051287_dd85625a04.jpg#2,0.0,0,3
2,1056338697_4f7d7ce270.jpg,1427391496_ea512cbe7f.jpg#2,0.0,0,3
3,1056338697_4f7d7ce270.jpg,2073964624_52da3a0fc4.jpg#2,0.0,0,3
4,1056338697_4f7d7ce270.jpg,2083434441_a93bc6306b.jpg#2,0.0,0,3


Описание колонок:
* `image` - Имя файла изображения;
* `query_id` - Идентификатор описания;
* `positive` - Доля людей, подтвердивших, что описание соответствует изображению;
* `n1` - Количество человек, подтвердивших, что описание соответствует изображению;
* `n2` - Количество человек, подтвердивших, что описание не соответствует изображению.

#### Оценка экспертов

In [240]:
expert_annotations_tsv = pd.read_csv(ROOT_FOLDER + '/to_upload/ExpertAnnotations.tsv', 
                                     sep='\t', 
                                     header=None, 
                                     names=['image', 'query_id', 'n1', 'n2', 'n3'])

expert_annotations_tsv.reset_index(inplace=True, drop=True)

expert_annotations_tsv.info()

expert_annotations_tsv.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image     5822 non-null   object
 1   query_id  5822 non-null   object
 2   n1        5822 non-null   int64 
 3   n2        5822 non-null   int64 
 4   n3        5822 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 227.5+ KB


,image,query_id,n1,n2,n3
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,1,1,1
1,1056338697_4f7d7ce270.jpg,2718495608_d8533e3ac5.jpg#2,1,1,2
2,1056338697_4f7d7ce270.jpg,3181701312_70a379ab6e.jpg#2,1,1,2
3,1056338697_4f7d7ce270.jpg,3207358897_bfa61fa3c6.jpg#2,1,2,2
4,1056338697_4f7d7ce270.jpg,3286822339_5535af6b93.jpg#2,1,1,2


Описание колонок:
* `image` - Имя файла изображения;
* `query_id` - Идентификатор описания;
* `n1` — оценка 1 эксперта;
* `n2` — оценка 2 эксперта;
* `n3` — оценка 3 эксперта.

Выполним обработку данных краудсорсинга:
* добавим колонку `n0` с агрегированной оценкой;
* добавим колонку `positive` с долей, подтвердивших, что описание соотвествует изображению.

Эксперты ставят оценки по шкале от `1` до `4`, где 
* `1` — изображение и запрос совершенно не соответствуют друг другу;
* `2` — запрос содержит элементы описания изображения, но в целом запрос тексту не соответствует; 
* `3` — запрос и текст соответствуют с точностью до некоторых деталей;
* `4` — запрос и текст соответствуют полностью.

___Примечание___: "положительными" будем считать оценки `3` и `4`

In [241]:
# выполним обработку данных краудсорсинга
def calc_avg(row):
    """
    Вычисление результата "голосования" способом большинства
    
    Параметры:
    ----------
    row: Series
    
    Результат:
    ----------
    int - число
    """
    array = [row['n1'], row['n2'], row['n3']]
    u, c = np.unique(array, return_counts=True)

    # находим самую частую оценку
    y = u[c == c.max()]
    # вычисляем долю найденных оценок
    percentages = dict(zip(u, c / len(array)))
    
    # "позитивными" будем считать, тех у кого оценка 3 или 4
    row['n0'] = y[0] if len(y) == 1 else 0
    row['positive'] = percentages[y[0]] if len(y) == 1 and y[0] in [3, 4] else 0
    
    return row

In [242]:
expert_annotations_tsv = expert_annotations_tsv.apply(calc_avg, axis=1)

In [243]:
# удалим те мнения экспертов, где они полностью разошлись
expert_annotations_n0 = expert_annotations_tsv[expert_annotations_tsv['n0'] == 0]

display(expert_annotations_n0.head())

expert_annotations_tsv.drop(expert_annotations_n0.index, inplace=True)

,image,query_id,n1,n2,n3,n0,positive
17,1082379191_ec1e53f996.jpg,1536774449_e16b1b6382.jpg#2,1,2,3,0,0.0
40,1119015538_e8e796281e.jpg,2534502836_7a75305655.jpg#2,2,3,4,0,0.0
47,1122944218_8eb3607403.jpg,3325497914_f9014d615b.jpg#2,2,3,4,0,0.0
168,1287475186_2dee85f1a5.jpg,2918769188_565dd48060.jpg#2,1,2,3,0,0.0
199,1329832826_432538d331.jpg,1536774449_e16b1b6382.jpg#2,1,2,3,0,0.0


In [244]:
expert_annotations_tsv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5696 entries, 0 to 5821
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   image     5696 non-null   object 
 1   query_id  5696 non-null   object 
 2   n1        5696 non-null   int64  
 3   n2        5696 non-null   int64  
 4   n3        5696 non-null   int64  
 5   n0        5696 non-null   int64  
 6   positive  5696 non-null   float64
dtypes: float64(1), int64(4), object(2)
memory usage: 356.0+ KB


### Объединение оценок

In [245]:
# для удобства установим индексы
crowd_annotations_tsv.set_index(['image', 'query_id'], inplace=True)
expert_annotations_tsv.set_index(['image', 'query_id'], inplace=True)

In [246]:
annotations = crowd_annotations_tsv[['positive']].merge(
    expert_annotations_tsv[['positive']], 
    how='outer', # используем это объединение, так как нужно соеденить все данные независимо от ключа
    left_index=True, 
    right_index=True, 
    suffixes=('_crowd', '_expert')
)

In [247]:
def join_positive_expert_priority(row):
    """
    Объединение оценок с приоритетом для экспертов
    
    Параметры:
    ----------
    row: Series
    
    Результат:
    ----------
    int - число
    """
    crowd = row['positive_crowd']
    expert = row['positive_expert']
    
    if expert >= 0:
        # важны только экспертные мнения
        return expert
    
    # если эксперты не давали оценку, а результат есть на кроудсорсинге
    if crowd >= 0 and expert != expert:
        return crowd
    
    return 0

In [248]:
annotations['positive'] = annotations.apply(join_positive_expert_priority, axis=1)

In [249]:
annotations.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 51268 entries, ('1056338697_4f7d7ce270.jpg', '1056338697_4f7d7ce270.jpg#2') to ('997722733_0cb5439472.jpg', '997722733_0cb5439472.jpg#2')
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   positive_crowd   47830 non-null  float64
 1   positive_expert  5696 non-null   float64
 2   positive         51268 non-null  float64
dtypes: float64(3)
memory usage: 1.4+ MB


In [250]:
train_dataset_csv.set_index(['image', 'query_id'], inplace=True)

In [251]:
train_dataset = train_dataset_csv.merge(annotations['positive'], how='left', left_index=True, right_index=True)

In [252]:
# проверяем наличие данных
train_dataset[train_dataset['positive'] > 0].sample(3)

,,query_text,positive
image,query_id,,
2985679744_75a7102aab.jpg,2985679744_75a7102aab.jpg#2,A man sitting on a subway .,1.000000
3121521593_18f0ec14f7.jpg,480505313_2dc686e5db.jpg#2,An older couple posing in front of a fountain ...,0.666667
3393926562_66cc01b001.jpg,448658518_eec0b648a6.jpg#2,A man dressed in black stands at a street corn...,0.666667


In [253]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5822 entries, ('1056338697_4f7d7ce270.jpg', '2549968784_39bfbe44f9.jpg#2') to ('968081289_cdba83ce2e.jpg', '2292406847_f366350600.jpg#2')
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   query_text  5822 non-null   object 
 1   positive    5767 non-null   float64
dtypes: float64(1), object(1)
memory usage: 322.9+ KB


## 2. Проверка данных

В некоторых странах, где работает ваша компания, действуют ограничения по обработке изображений: поисковым сервисам и сервисам, предоставляющим возможность поиска, запрещено без разрешения родителей или законных представителей предоставлять любую информацию, в том числе, но не исключительно тексты, изображения, видео и аудио, содержащие описание, изображение или запись голоса детей. Ребёнком считается любой человек, не достигший 16 лет.

В вашем сервисе строго следуют законам стран, в которых работают. Поэтому при попытке посмотреть изображения, запрещённые законодательством, вместо картинок показывается дисклеймер:

> This image is unavailable in your country in compliance with local laws
> 

Однако у вас в PoC нет возможности воспользоваться данным функционалом. Поэтому все изображения, которые нарушают данный закон, нужно удалить из обучающей выборки.

In [254]:
stop_words = ['baby', 'child', 'little boy', 'little girl', 'kid', 'young girl', 'young boy', 'boy', 'girl']

In [255]:
def ban(txt):
    """
    Определение наличие стоп-слов
    
    Параметры:
    ----------
    txt: string - текст для анализа
    
    Результат:
    ----------
    boolean - результат обработки
    """
    txt = txt.lower()
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(txt)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_tokens])
    
    for word in stop_words:
        word_len = len(word.split())
        
        if word_len > 1:
            if word in txt:
                return True
        elif word_len == 1:
            if word in txt.split():
                return True
        
    return False

In [256]:
train_dataset['ban'] = train_dataset['query_text'].apply(ban)

train_dataset[train_dataset['ban'] == True].info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1200 entries, ('1056338697_4f7d7ce270.jpg', '2549968784_39bfbe44f9.jpg#2') to ('757046028_ff5999f91b.jpg', '2061144717_5b3a1864f0.jpg#2')
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   query_text  1200 non-null   object 
 1   positive    1185 non-null   float64
 2   ban         1200 non-null   bool   
dtypes: bool(1), float64(1), object(1)
memory usage: 104.7+ KB


In [257]:
train_dataset.drop(train_dataset[train_dataset.ban == True].index, inplace=True)

## 3. Векторизация изображений

Перейдём к векторизации изображений.

Самый примитивный способ — прочесть изображение и превратить полученную матрицу в вектор. Такой способ нам не подходит: длина векторов может быть сильно разной, так как размеры изображений разные. Поэтому стоит обратиться к свёрточным сетям: они позволяют "выделить" главные компоненты изображений. Как это сделать? Нужно выбрать какую-либо архитектуру, например ResNet-18, посмотреть на слои и исключить полносвязные слои, которые отвечают за конечное предсказание. При этом можно загрузить модель данной архитектуры, предварительно натренированную на датасете ImageNet.

## 4. Векторизация текстов

Следующий этап — векторизация текстов. Вы можете поэкспериментировать с несколькими способами векторизации текстов:

- tf-idf
- word2vec
- \*трансформеры (например Bert)

\* — если вы изучали трансформеры в спринте Машинное обучение для текстов.


## 5. Объединение векторов

Подготовьте данные для обучения: объедините векторы изображений и векторы текстов с целевой переменной.

## 6. Обучение модели предсказания соответствия

Для обучения разделите датасет на тренировочную и тестовую выборки. Простое случайное разбиение не подходит: нужно исключить попадание изображения и в обучающую, и в тестовую выборки.
Для того чтобы учесть изображения при разбиении, можно воспользоваться классом [GroupShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupShuffleSplit.html) из библиотеки sklearn.model_selection.

Код ниже разбивает датасет на тренировочную и тестовую выборки в пропорции 7:3 так, что строки с одинаковым значением 'group_column' будут содержаться либо в тестовом, либо в тренировочном датасете.

```
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=42)
train_indices, test_indices = next(gss.split(X=df.drop(columns=['target']), y=df['target'], groups=df['group_column']))
train_df, test_df = df.loc[train_indices], df.loc[test_indices]

```

Какую модель использовать — выберите самостоятельно. Также вам предстоит выбрать метрику качества либо реализовать свою.

## 7. Тестирование модели

Настало время протестировать модель. Для этого получите эмбеддинги для всех тестовых изображений из папки `test_images`, выберите случайные 10 запросов из файла `test_queries.csv` и для каждого запроса выведите наиболее релевантное изображение. Сравните визуально качество поиска.

## 8. Выводы

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Исследовательский анализ данных выполнен
- [ ]  Проверены экспертные оценки и краудсорсинговые оценки
- [ ]  Из датасета исключены те объекты, которые выходят за рамки юридических ограничений
- [ ]  Изображения векторизованы
- [ ]  Текстовые запросы векторизованы
- [ ]  Данные корректно разбиты на тренировочную и тестовую выборки
- [ ]  Предложена метрика качества работы модели
- [ ]  Предложена модель схожести изображений и текстового запроса
- [ ]  Модель обучена
- [ ]  По итогам обучения модели сделаны выводы
- [ ]  Проведено тестирование работы модели
- [ ]  По итогам тестирования визуально сравнили качество поиска